In [1]:
from __future__ import print_function, division
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.applications import densenet
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
#from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import tensorflow as tf
import keras

from sklearn import preprocessing
import numpy as np
from numpy import expand_dims
from imutils import paths
from sklearn.model_selection import train_test_split
import cv2
import os
import tensorflow as tf
import os
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

INIT_LR = 0.0001
EPOCHS = 35
BS = 16
num_folds= 5
verbosity = 1

Using TensorFlow backend.


# 데이터 파일 불러오기 및 전처리 과정

In [2]:
imagePaths = list(paths.list_images("under_2weeks_maxillary"))
data = []
labels = []

for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (56, 56), interpolation=cv2.INTER_AREA)
    # image = image.reshape(3, 224, 224)
    data.append(image)
    labels.append(label)
    # convert the data and labels to NumPy arrays while scaling the pixel
    # intensities to the range [0, 1]
data = np.array(data) / 255.0
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
#labels = 1-lb.fit_transform(labels)
labels = to_categorical(labels)
(X_train2, X_test2, y_train2, y_test2) = train_test_split(data, labels, test_size=0.2, stratify=labels,
                                                      random_state=42)
#$$$$X_train = np.concatenate((X_train, X_test), axis=0)
#$$$$y_train = np.concatenate((y_train, y_test), axis=0)
#$$$$X_train = expand_dims(X_train, axis=-1)
#$$$$X_train = (X_train.astype(np.float32) - 127.5) / 127.5
#$$$$y_train = y_train.reshape(-1, 1)

# Pre-trained CheXNet 모델 weight 저장

In [3]:
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.layers import Dense, Input
from keras.models import Model, load_model
from keras.utils import print_summary

# chexNet weights
# https://github.com/brucechou1983/CheXNet-Keras
chexnet_weights = 'brucechou1983_CheXNet_Keras_0.3.0_weights.h5'

# chexnet class names
chexnet_class_index_to_name = [
    'Atelectasis',  # 0
    'Cardiomegaly',  # 1
    'Effusion',  # 2
    'Infiltration',  # 3
    'Mass',  # 4
    'Nodule',  # 5
    'Pneumonia',  # 6
    'Pneumothorax',  # 7
    'Consolidation',  # 8
    'Edema',  # 9
    'Emphysema',  # 10
    'Fibrosis',  # 11
    'Pleural_Thickening',  # 12
    'Hernia',  # 13
]

# chexnet class indexes
chexnet_class_name_to_index = {
    'Atelectasis': 0,
    'Cardiomegaly': 1,
    'Effusion': 2,
    'Infiltration': 3,
    'Mass': 4,
    'Nodule': 5,
    'Pneumonia': 6,
    'Pneumothorax': 7,
    'Consolidation': 8,
    'Edema': 9,
    'Emphysema': 10,
    'Fibrosis': 11,
    'Pleural_Thickening': 12,
    'Hernia': 13,
}

def chexnet_preprocess_input(value):
    return preprocess_input(value)


def get_chexnet_model():
    input_shape = (56, 56, 3) #input image shape 설정
    img_input = Input(shape=input_shape)
    base_weights = 'imagenet'

    # create the base pre-trained model
    base_model = DenseNet121(
        include_top=False,
        input_tensor=img_input,
        input_shape=input_shape,
        weights=base_weights,
        pooling='avg'
    )

    x = base_model.output
    # add a logistic layer -- let's say we have 14 classes
    predictions = Dense(
        14,
        activation='sigmoid',
        name='predictions')(x)

    # this is the model we will use
    model = Model(
        inputs=img_input,
        outputs=predictions,
    )

    # load chexnet weights
    model.load_weights(chexnet_weights)

    # return model
    return base_model, model


if __name__ == '__main__':
    '''
    What this script is doing is that
    1. It first loads a chexnet nodel (DenseNet121),
       where the weight is from https://github.com/brucechou1983/CheXNet-Keras
    2. It then print a summary of the model architecture
    '''
    # get model
    base_model, model = get_chexnet_model()
    # print a model summary
    print_summary(model)
    
input_shape = (56, 56, 3) #input image shape에 맞게 설정
model.build(input_shape)
model.save('CheXNet_TransferLearning_PNS_GAN.h5') #Pre-trained CheXNet model 저장 (저장 경로 지정)














__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 56, 56, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 62, 62, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 28, 28, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 28, 28, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________________

# Output Layer 수정

In [4]:
from keras.callbacks import ModelCheckpoint
model.load_weights('CheXNet_TransferLearning_PNS_GAN.h5') #transfer learning을 위해 pre-trained chexnet 업로드

from keras import layers

model.layers.pop()
model.layers[-2].outbound_nodes = []
model.outputs = [model.layers[-2].output]
output = model.get_layer('avg_pool').output
#output = Dropout(0.2)(output)
predictions = Dense(2, activation='sigmoid')(output) #다중 분류의 경우 softmax
new_model = Model(model.input, predictions)
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 56, 56, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 62, 62, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 28, 28, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 28, 28, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

# 모델 학습 

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
new_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc', 'mse'])

H = new_model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    #validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS)

new_model.save_weights('under2weeks_CheXNet_TransferLearning_20200603_.h5')

# 모델 평가

In [ ]:
new_model.load_weights('best_model_chexGAN_imbalanced_4.hdf5')
new_model.evaluate(X_test, y_test, verbose=1)

predIdxs = new_model.predict(X_test, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(y_test.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(y_test.argmax(axis=1), predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))